# Welcome to the Trading Simulator Bot! 🚀

This bot is designed to simulate a trading investment strategy using real-time cryptocurrency data.  
The main purpose is to allow you to explore how different strategies would impact your investment  
in the volatile world of crypto. Currently, the bot offers two strategies: Moving Average (MA) and  
Relative Strength Index (RSI). But don't worry – the code is built with scalability in mind, so you  
can easily add more strategies in the future as you explore different trading techniques.

Once you set your investment amount, the bot will track your balance based on the chosen strategy  
and the market movements of Bitcoin (BTC). You’ll be able to see how your investment performs,  
whether you’re buying during bullish trends or selling during bearish moments – all in the safety  
of a simulation environment. So, get ready to test your trading knowledge and see how your strategy  
performs in real-world market conditions!


In [ ]:
import asyncio
import json
import requests
from datetime import datetime
import plotly.graph_objects as go
import time

# Function to get the current BTC/USDT price from Binance
def get_current_price():
    url = "https://api.binance.com/api/v3/ticker/price?symbol=BTCUSDT"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad status codes
        data = response.json()
        return float(data['price'])  # Bitcoin price in USD
    except requests.exceptions.RequestException as e:
        print(f"Error fetching price: {e}")
        return None

# Function to calculate the Moving Average
def moving_average(prices, period=10):
    if len(prices) < period:
        return None
    return sum(prices[-period:]) / period

# Function to calculate the RSI
def relative_strength_index(prices, period=14):
    if len(prices) < period:
        return None
    gains = 0
    losses = 0
    for i in range(1, period+1):
        change = prices[-i] - prices[-i-1]
        if change > 0:
            gains += change
        elif change < 0:
            losses += abs(change)
    
    avg_gain = gains / period
    avg_loss = losses / period
    
    if avg_loss == 0:
        return 100  # If there are no losses, RSI is 100
    rs = avg_gain / avg_loss
    return 100 - (100 / (1 + rs))

# Base class for trading strategies
class TradingStrategy:
    def should_buy(self, prices, current_balance, investment_amount):
        raise NotImplementedError

    def should_sell(self, prices, current_position):
        raise NotImplementedError

# Moving Average Strategy
class MovingAverageStrategy(TradingStrategy):
    def should_buy(self, prices, current_balance, investment_amount):
        ma = moving_average(prices)
        current_price = prices[-1]
        return ma and current_price > ma and current_balance >= investment_amount

    def should_sell(self, prices, current_position):
        ma = moving_average(prices)
        current_price = prices[-1]
        return ma and current_price < ma and current_position > 0

# RSI Strategy
class RSIStrategy(TradingStrategy):
    def should_buy(self, prices, current_balance, investment_amount):
        rsi = relative_strength_index(prices)
        return rsi and rsi < 30 and current_balance >= investment_amount

    def should_sell(self, prices, current_position):
        rsi = relative_strength_index(prices)
        return rsi and rsi > 70 and current_position > 0

# Create the trading bot class
class TradingBot:
    def __init__(self, initial_balance, strategy):
        self.initial_balance = initial_balance
        self.current_balance = initial_balance
        self.current_position = 0  # Amount of BTC owned
        self.entry_price = None  # Price at which the purchase was made
        self.transactions = []  # List to track transactions
        self.strategy = strategy
        self.price_history = []  # Price history for calculating indicators

    def simulate_trade(self, investment_amount):
        """Simulate a trade based on the selected strategy."""
        if len(self.price_history) < 2:  # Need at least 2 prices to calculate indicators
            return

        # Check if the buy/sell strategy should be applied
        if self.strategy.should_buy(self.price_history, self.current_balance, investment_amount):
            self.buy(self.price_history[-1], investment_amount)
        elif self.strategy.should_sell(self.price_history, self.current_position):
            self.sell(self.price_history[-1])

    def buy(self, price, investment_amount):
        """Execute a purchase."""
        quantity = investment_amount / price  # Amount of BTC bought
        self.current_balance -= investment_amount  # Deduct amount from account
        self.current_position += quantity  # Add the purchased BTC to balance
        self.entry_price = price
        self.transactions.append({
            'type': 'buy',
            'quantity': quantity,
            'price': price,
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        })
        
        # Print the alert
        print(f"Purchase made! BTC balance: {self.current_position} | USD balance: {self.current_balance}")

    def sell(self, price):
        """Execute a sale."""
        sale_amount = self.current_position * price
        self.current_balance += sale_amount
        self.current_position = 0
        self.transactions.append({
            'type': 'sell',
            'quantity': self.current_position,
            'price': price,
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        })
        print(f"Sale made! BTC balance: {self.current_position} | USD balance: {self.current_balance}")

    def save_investment(self, filename="investments.json"):
        """Save investment data in a JSON file."""
        data = {
            "initial_balance": self.initial_balance,
            "current_balance": self.current_balance,
            "current_position": self.current_position,
            "entry_price": self.entry_price,
            "transactions": self.transactions
        }
        with open(filename, "w") as file:
            json.dump(data, file, indent=4)

    def get_portfolio(self):
        """Return the total balance in USD, considering the BTC balance."""
        if self.current_position > 0:
            current_price = get_current_price()  # Current BTC price
            return self.current_balance + self.current_position * current_price
        return self.current_balance

# Function to update the graph in real-time using Plotly
async def update_graph(fig, prices, timestamps):
    while True:
        # Update the graph data
        fig.data = []  # Clear previous data
        fig.add_trace(go.Scatter(x=timestamps, y=prices, mode='lines', name='BTC Price'))
        fig.update_layout(
            title="BTC Price Trend Over Time",
            xaxis_title="Time",
            yaxis_title="Price in USD",
            template="plotly_dark"
        )
        fig.show()
        await asyncio.sleep(0.1)  # Small delay to avoid overwhelming the system

# Main function for trading and updating graphs
async def main():
    # Ask the user to choose a strategy
    print("Select a strategy: ")
    print("1. Moving Average (ma)")
    print("2. RSI (rsi)")
    choice = input("Enter 'ma' for Moving Average or 'rsi' for RSI: ").strip().lower()

    # Ask the user for initial balance
    initial_balance = float(input("Enter your initial balance in USD: ").strip())

    # Ask the user how much money they want to invest per trade
    investment_amount = float(input("Enter the amount you want to invest per trade in USD: ").strip())

    # Create the bot with the chosen strategy
    if choice == "ma":
        strategy = MovingAverageStrategy()
    elif choice == "rsi":
        strategy = RSIStrategy()
    else:
        print("Invalid strategy choice. Exiting.")
        return

    bot = TradingBot(initial_balance=initial_balance, strategy=strategy)

    prices = []
    timestamps = []
    
    # Initialize the graph
    fig = go.Figure()
    fig.show()

    # Run the bot for a fixed number of trades (e.g., 50 trades)
    max_trades = 50
    trade_count = 0

    # Start the graph update task
    graph_task = asyncio.create_task(update_graph(fig, prices, timestamps))

    while trade_count < max_trades:
        # Fetch the price 3 times per second (~333 ms)
        current_price = get_current_price()  # Current BTC price from Binance
        if current_price is not None:
            prices.append(current_price)
            timestamps.append(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

            # Simulate a trade (independent of price updates)
            bot.simulate_trade(investment_amount)
            print(f"Trade {trade_count + 1} completed. BTC balance: {bot.current_position} | USD balance: {bot.current_balance}")
            trade_count += 1

        await asyncio.sleep(0.333)  # Wait ~333 ms between price updates

    print(f"Simulation completed after {max_trades} trades.")
    graph_task.cancel()  # Stop the graph update task

# Run the bot using nest_asyncio to integrate with the IPython event loop
import nest_asyncio
nest_asyncio.apply() 
asyncio.run(main())